# Preparation
The data on HDB resale property transactions comes in 4 separate datasets (link:
https://data.gov.sg/dataset/resale-flat-prices) differentiated by the date of the transaction:

- 1990 – 1999
- 2000 – Feb 2012
- Mar 2012 – Dec 2014
- Jan 2015 onwards

Prior to attempting the questions below, please merge the datasets together.

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [20]:
df = pd.read_csv('hdb_resale.csv')

/Users/Ju/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
# data source: https://data.gov.sg/dataset/resale-flat-prices
query_string=[
    # Jan 2017 onwards
    'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=103555',
    # Jan 2015 to Dec 2016
    'https://data.gov.sg/api/action/datastore_search?resource_id=1b702208-44bf-4829-b620-4615ee19b57c&limit=37153',
    # Mar 2012 to Dec 2014
    'https://data.gov.sg/api/action/datastore_search?resource_id=83b2fc37-ce8c-4df4-968b-370fd818138b&limit=52203',
    #2000 to Feb 2012
    'https://data.gov.sg/api/action/datastore_search?resource_id=8c00bf08-9124-479e-aeca-7cc411d884c4&limit=369651',
    #1990 to 1999
    'https://data.gov.sg/api/action/datastore_search?resource_id=adbbddd3-30e2-445f-a123-29bee150a6fe&limit=287196'
]

Using API provided by data.gov, a function is made to compile all the data and to form one merged sheet

In [24]:
def hdb_api(query_string):
    all_df = []
    for url in query_string:
        resp = requests.get(url)
        
        #Convert JSON into Python Object 
        data = json.loads(resp.content)
        
        # find length of data and columns
        hdb_price_dict_records = data['result']['records']
        col = len(hdb_price_dict_records[0])
        
        # Next we need to feed our JSON data into dataframe. 
        # We will access the 'records' key:value pairs of the python dictionary. 
        # We will then map the list into a dataframe.
        town = []
        flat_type = []
        flat_model = []
        floor_area_sqm = []
        street_name = []
        resale_price = []
        month = []
        remaining_lease = []
        lease_commence_date = []
        storey_range = []
        _id = []
        block = []

        for i in range(len(hdb_price_dict_records)):
            town.append(hdb_price_dict_records[i]['town'])
            flat_type.append(hdb_price_dict_records[i]['flat_type'])
            flat_model.append(hdb_price_dict_records[i]['flat_model'])
            floor_area_sqm.append(hdb_price_dict_records[i]['floor_area_sqm'])
            street_name.append(hdb_price_dict_records[i]['street_name'])
            resale_price.append(hdb_price_dict_records[i]['resale_price'])
            month.append(hdb_price_dict_records[i]['month'])
            lease_commence_date.append(hdb_price_dict_records[i]['lease_commence_date'])
            storey_range.append(hdb_price_dict_records[i]['storey_range'])
            _id.append(hdb_price_dict_records[i]['_id'])
            block.append(hdb_price_dict_records[i]['block'])
            if col >11:
                remaining_lease.append(hdb_price_dict_records[i]['remaining_lease'])

        # append all_df to be combined later
        if col >11:
            all_df.append(pd.DataFrame({
                'town': town,
                'flat_type': flat_type,
                'flat_model': flat_model,
                'floor_area_sqm': floor_area_sqm,
                'street_name': street_name,
                'resale_price': resale_price,
                'month': month,
                'remaining_lease': remaining_lease,
                'lease_commence_date': lease_commence_date,
                'storey_range': storey_range,
                '_id': _id,
                'block': block
            }))
        else:
            all_df.append(pd.DataFrame({
                'town': town,
                'flat_type': flat_type,
                'flat_model': flat_model,
                'floor_area_sqm': floor_area_sqm,
                'street_name': street_name,
                'resale_price': resale_price,
                'month': month,
                'lease_commence_date': lease_commence_date,
                'storey_range': storey_range,
                '_id': _id,
                'block': block
            }))
        
    return pd.concat(all_df,ignore_index=True)

In [25]:
df = hdb_api(query_string)

In [26]:
# Verify that there is no missing entries (849758 entries)
len(df)

849758

## Getting to know the dataset
The HDB resale data can be retrieved from data.gov.sg. Below is an example of the columns available from the [`HDB_resale.csv`]('HDB_resale.csv') dataset:

|Feature| Description |
|--|--|
| Month | Given in the format of year-month. We may retrieve the year data from this column, which may be useful when analysing the time trend for HDB resale price. |
| Town | Town location should be one of the key factors affecting HDB resale price — we are generally expecting an HDB flat in Orchard has a much higher resale price than Yishun given the same flat type.|
|Flat Type| There are 7 different kinds of flat types: 1 Room, 2 Room, 3 Room, 4 Room, 5 Room, EC and Multi-generation. Among which the 4 Room HDB flats are the most popular ones in Singapore. We may consider using 4 Room data samples to construct the model. |
|Storey Range| This column is given as a string rather than numbers, we may need to do some data munging accordingly if we want to use it to build the model. |
|Flat Model| Similarly, there are plenty of different flat models out there(35 different types). This factor would play an important role in the overall flat price. E.g., the DBSS (Design, Build and Sell Scheme) flats would have a higher resale price considering it allows buyers to design the HDB based on their own style. |
|Remaining Lease| Singapore HDB has a lease of 99 years. This column data has quite some NULL values, and it is calculated based on different years. We may need to adjust this column data accordingly when building the model.|

## Datatypes

In [27]:
df.dtypes

town                   object
flat_type              object
flat_model             object
floor_area_sqm         object
street_name            object
resale_price           object
month                  object
remaining_lease        object
lease_commence_date    object
storey_range           object
_id                     int64
block                  object
dtype: object

In [28]:
df.columns.to_list()

['town',
 'flat_type',
 'flat_model',
 'floor_area_sqm',
 'street_name',
 'resale_price',
 'month',
 'remaining_lease',
 'lease_commence_date',
 'storey_range',
 '_id',
 'block']

In [29]:
df.head(2)

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,WOODLANDS,5 ROOM,Model A,134,MARSILING RD,350000,2017-03,77 years 01 month,1995,04 TO 06,1,139
1,WOODLANDS,5 ROOM,Standard,120,MARSILING DR,355000,2017-03,57 years 10 months,1976,16 TO 18,2,9


The following column datatypes need to be handled: 

|Feature| Data Type |
|--|--|
|'_id' | should be dropped because there it has no meaning to the database other than identify itself in its .csv file |
|'storey_range' | to two columns lower and upper storeys as separate columns |
| 'floor_area_sqm' 'resale_price'  'storey_lower' 'storey_upper' 'block_num' | to numeric |
| 'month','lease_commence_date'| to date|
|'remaining_lease'| has many ms may not be used so not amended|

In [30]:
df.drop(columns='_id',inplace=True)

In [31]:
df['storey_lower'] = df['storey_range'].apply(lambda x: re.findall('\d+',x)[0])
df['storey_upper'] = df['storey_range'].apply(lambda x: re.findall('\d+',x)[1])
df['block_num'] = df['block'].apply(lambda x: re.findall('\A\d+',x)[0])

In [32]:
# convert numeric columns
for x in ['floor_area_sqm','resale_price','storey_lower', 'storey_upper', 'block_num']:
    df[x] = pd.to_numeric(df[x])

In [33]:
# change to time format for date columns
for x in ['month','lease_commence_date']:
    df[x] = pd.to_datetime(df[x])

In [34]:
df['storey_ave'] = (df['storey_lower'] + df['storey_upper'])/2

In [35]:
numeric_feat =[col for col in df.columns if df[col].dtypes != 'O'and col not in ['_id']]
discrete_feat = [col for col in numeric_feat if len(df[col].unique())<25 and col not in ['_id']]
continuous_feat = [col for col in numeric_feat if col not in discrete_feat and col not in ['_id']]
categorical_feat = [col for col in df.columns if df[col].dtypes == 'O']

In [17]:
feature_types = [numeric_feat,discrete_feat,continuous_feat,categorical_feat]
feature_label = ['numeric','discrete','continuous','categorical']

for feat,ls in zip(feature_label,feature_types):
    print(f'{feat} features: {len(ls)}, \n{ls}\n')

numeric features: 8, 
['floor_area_sqm', 'resale_price', 'month', 'lease_commence_date', 'storey_lower', 'storey_upper', 'block_num', 'storey_ave']

discrete features: 3, 
['storey_lower', 'storey_upper', 'storey_ave']

continuous features: 5, 
['floor_area_sqm', 'resale_price', 'month', 'lease_commence_date', 'block_num']

categorical features: 7, 
['town', 'flat_type', 'flat_model', 'street_name', 'remaining_lease', 'storey_range', 'block']



## Dealing with duplicates

There is some duplicated data in this set. However, given the unspecific descriptors of the data it can be assumed that they are distinct purchases. So duplicates are not removed

In [36]:
df[df.duplicated(keep=False)].head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,block,storey_lower,storey_upper,block_num,storey_ave
235,ANG MO KIO,4 ROOM,New Generation,91.0,ANG MO KIO AVE 1,470000.0,2017-04-01,63 years 10 months,1982-01-01,04 TO 06,335,4,6,335,5.0
245,ANG MO KIO,4 ROOM,New Generation,91.0,ANG MO KIO AVE 1,470000.0,2017-04-01,63 years 10 months,1982-01-01,04 TO 06,335,4,6,335,5.0
275,BEDOK,3 ROOM,New Generation,67.0,BEDOK NTH ST 3,288000.0,2017-04-01,61 years 05 months,1979-01-01,01 TO 03,510,1,3,510,2.0
276,BEDOK,3 ROOM,New Generation,67.0,BEDOK NTH ST 3,288000.0,2017-04-01,61 years 05 months,1979-01-01,01 TO 03,510,1,3,510,2.0
335,BEDOK,4 ROOM,Model A,108.0,JLN TENAGA,470000.0,2017-04-01,78 years 07 months,1996-01-01,04 TO 06,655,4,6,655,5.0


## Review Categories

'flat_type', 'flat_model' needs to be conditioned to remove redundancies

In [37]:
for feat in categorical_feat:
    print(f'======={feat}=======')
    print(df[feat].unique())

=======town=======
['WOODLANDS' 'YISHUN' 'ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK'
 'PUNGGOL' 'BUKIT MERAH' 'BUKIT PANJANG' 'BUKIT TIMAH' 'CENTRAL AREA'
 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG' 'HOUGANG' 'JURONG EAST'
 'JURONG WEST' 'KALLANG/WHAMPOA' 'MARINE PARADE' 'PASIR RIS' 'QUEENSTOWN'
 'SEMBAWANG' 'SENGKANG' 'SERANGOON' 'TAMPINES' 'TOA PAYOH' 'LIM CHU KANG']
=======flat_type=======
['5 ROOM' 'EXECUTIVE' '3 ROOM' '4 ROOM' '2 ROOM' '1 ROOM'
 'MULTI-GENERATION' 'MULTI GENERATION']
=======flat_model=======
['Model A' 'Standard' 'Improved' 'Premium Apartment' 'Apartment'
 'Maisonette' 'New Generation' 'Simplified' 'DBSS' 'Model A2' 'Type S1'
 'Type S2' 'Model A-Maisonette' 'Adjoined flat' 'Improved-Maisonette'
 'Terrace' 'Premium Maisonette' 'Multi Generation'
 'Premium Apartment Loft' '2-room' 'IMPROVED' 'NEW GENERATION' 'MODEL A'
 'STANDARD' 'SIMPLIFIED' 'MODEL A-MAISONETTE' 'APARTMENT' 'MAISONETTE'
 'TERRACE' '2-ROOM' 'IMPROVED-MAISONETTE' 'MULTI GENERATION'
 'PREMIUM APARTMENT']
==

['77 years 01 month' '57 years 10 months' '58 years' '79 years 07 months'
 '86 years' '71 years 08 months' '79 years 06 months' '85 years 10 months'
 '85 years' '78 years 01 month' '63 years 02 months' '79 years 09 months'
 '79 years' '78 years 05 months' '80 years' '82 years 02 months'
 '80 years 09 months' '82 years 04 months' '82 years 08 months'
 '80 years 03 months' '78 years 07 months' '79 years 03 months'
 '80 years 10 months' '78 years 10 months' '82 years 06 months'
 '74 years 11 months' '84 years 11 months' '83 years 01 month' '78 years'
 '79 years 05 months' '82 years' '79 years 04 months' '84 years 10 months'
 '79 years 01 month' '80 years 08 months' '78 years 03 months'
 '68 years 03 months' '82 years 03 months' '67 years 09 months'
 '83 years 07 months' '81 years' '75 years 10 months' '77 years'
 '77 years 05 months' '76 years 01 month' '71 years 02 months'
 '66 years 08 months' '66 years 06 months' '74 years 03 months'
 '68 years 11 months' '66 years 10 months' '70 years

In [38]:
# focus on flat_type, flat_model which have redundant titles
# focus on flat_type, flat_model which have redundant titles
def labeler(feat):
    temp = []
    for value in df[feat].unique():
        if value != ' '.join(re.findall( r'\w+', value.upper())):
                df[feat].replace(value,' '.join(re.findall( r'\w+', value.upper())),inplace=True)

In [39]:
for feat in ['flat_type', 'flat_model']:
    labeler(feat)

### Remaining Lease

Assign the remaining_lease based on 99 year lease for each HDB

In [54]:
df['remaining_lease'] = df['lease_commence_date'].dt.year + 99 - df['month'].dt.year

### Create a .csv of raw data

In [22]:
df.to_csv('hdb_resale.csv',index=False)

## Getting location data from OneMap

In [20]:
# Now, the next part gets a little tricky. We need to find the nearest MRT station the HDB unit is located in using their geo-location coordinates (Latitude, Longitude). We will use another API to achieve this - the OneMap API.

# First let's create a list of all the MRT stations in Singapore. 
# Since MRT stations change relatively slowly overtime, we can leverage a static list. 
# I obtained the data from Wikipedia, which also provides data on upcoming MRT stations.
# We will only consider existing MRT stations for now.
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station'
]


# In[10]:


list_of_shopping_mall = [
    '100 AM',
    '313@Somerset',
    'Aperia',
    'Balestier Hill Shopping Centre',
    'Bugis Cube',
    'Bugis Junction',
    'Bugis+',
    'Capitol Piazza',
    'Cathay Cineleisure Orchard',
    'City Gate',
    'City Square Mall',
    'CityLink Mall',
    'Clarke Quay Central',
    'Duo',
    'Far East Plaza',
    'Funan',
    'Great World City',
    'HDB Hub',
    'Holland Village Shopping Centre',
    'ION Orchard',
    'Junction 8',
    'Knightsbridge',
    'Liang Court',
    'Liat Towers',
    'Lucky Plaza',
    'Marina Bay Financial Centre Tower 3',
    'Marina Bay Link Mall',
    'Marina Bay Sands',
    'Marina One',
    'Marina Square',
    'Midpoint Orchard',
    'Millenia Walk',
    'Mustafa Centre',
    'Ngee Ann City',
    'Orchard Central',
    'Orchard Gateway',
    'Orchard Plaza',
    'Orchard Shopping Centre',
    'Palais Renaissance',
    'Peoples Park Centre',
    'People"s Park Complex',
    'Plaza Singapura',
    'PoMo',
    'Raffles City',
    'Scotts Square',
    'Serangoon Plaza',
    'Shaw House and Centre',
    'Sim Lim Square',
    'Singapore Shopping Centre',
    'Square 2',
    'Suntec City',
    'Tanglin Mall',
    'Tangs',
    'Tanjong Pagar Centre',
    'Tekka Centre',
    'The Centrepoint',
    'The Paragon',
    'The Poiz [2]',
    'The Shoppes at Marina Bay Sands',
    'The South Beach',
    'Thomson Plaza',
    'United Square, The Kids Learning Mall',
    'Velocity',
    'Wheelock Place',
    'Wisma Atria',
    'Zhongshan Mall',
    '112 Katong',
    'Bedok Mall',
    'Bedok Point',
    'Century Square',
    'Changi Airport',
    'Changi City Point',
    'City Plaza',
    'Djitsun Mall Bedok',
    'Downtown East',
    'East Village',
    'Eastpoint Mall',
    'Elias Mall',
    'Kallang Wave Mall',
    'Katong Square',
    'Katong V',
    'KINEX (formerly One KM Mall)',
    'Leisure Park Kallang',
    'Loyang Point',
    'Our Tampines Hub',
    'Parkway Parade',
    'Paya Lebar Square',
    'PLQ Mall',
    'Singapore Post Centre',
    'Tampines 1',
    'Tampines Mall',
    'The Flow',
    'White Sands',
    '888 Plaza',
    'Admiralty Place',
    'AMK Hub',
    'Beauty World Centre',
    'Beauty World Plaza',
    'Broadway Plaza',
    'Buangkok Square',
    'Bukit Panjang Plaza',
    'Bukit Timah Plaza',
    'Causeway Point',
    'Compass One',
    'Djitsun Mall',
    'Fajar Shopping Centre',
    'Greenridge Shopping Centre',
    'Greenwich V',
    'Heartland Mall',
    'Hillion Mall',
    'HillV2',
    'Hougang 1',
    'Hougang Green Shopping Mall',
    'Hougang Mall',
    'Jubilee Square',
    'Junction 10',
    'Junction 9',
    'Keat Hong Shopping Centre',
    'KKH The Retail Mall',
    'Limbang Shopping Centre',
    'Lot One',
    'Marsiling Mall',
    'myVillage @ Serangoon',
    'NEX',
    'North East',
    'North West',
    'Northpoint City',
    'Oasis Terraces',
    'Punggol Plaza',
    'Rail Mall',
    'Rivervale Mall',
    'Rivervale Plaza',
    'Sembawang Shopping Centre',
    'Sun Plaza',
    'Sunshine Place',
    'Teck Whye Shopping Centre',
    'The Midtown',
    'The Seletar Mall',
    'Upper Serangoon Shopping Centre',
    'Waterway Point',
    'West Mall',
    'Wisteria Mall',
    'Woodlands Mart',
    'Yew Tee Point',
    'Yew Tee Shopping Centre',
    'Yew Tee Square',
    'Alexandra Retail Centre',
    'HarbourFront Centre',
    'VivoCity',
    '321 Clementi',
    'Alexandra Central',
    'Anchorpoint',
    'Big Box',
    'Boon Lay Shopping Centre',
    'Fairprice Hub',
    'Gek Poh Shopping Centre',
    'Grantral Mall',
    'IMM',
    'JCube',
    'Jem',
    'Jurong Point',
    'OD Mall',
    'Pioneer Mall',
    'Queensway Shopping Centre',
    'Rochester Mall',
    'Taman Jurong Shopping Centre',
    'The Clementi Mall',
    'The Star Vista',
    'Tiong Bahru Plaza',
    'West Coast Plaza',
    'Westgate Mall',
]



In [39]:
# We will use the OneMap API to obtain the (lat, long) coordinates of each MRT station.
# Obtaining MRT coordinates in Singapore
mrt_building = []
mrt_lat = []
mrt_long = []


for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)
    
    if data_mrt['found'] != 0:
        mrt_building.append(data_mrt["results"][0]["BUILDING"])
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_building.append('NotFound')
        mrt_lat.append('NotFound')
        mrt_long.append('NotFound')
        print ("No Results")

# Store this information in a dataframe
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'Building': mrt_building,
    'Latitude': mrt_lat,
    'Longitude': mrt_long
})

Jurong East MRT Station,Lat: 1.33315281585758 Long: 103.742286332403
Bukit Batok MRT Station,Lat: 1.34903331201636 Long: 103.749566478309
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39753506936297 Long: 103.747405150236
Kranji MRT Station,Lat: 1.42508698073648 Long: 103.762137459497
Marsiling MRT Station,Lat: 1.43252114855026 Long: 103.774074641403
Woodlands MRT Station,Lat: 1.43605761708128 Long: 103.787938777173
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44905082158502 Long: 103.820046140211
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42944308477331 Long: 103.835005047246
Khatib MRT Station,Lat: 1.41738337009565 Long: 103.832979908243
Yio Chu Kang MRT Station,Lat: 1.38175587099132 Long: 103.84494727118
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.849558

In [40]:
# Obtaining Mall Coordinates in Singapore
mall_name = []
mall_roadname = []
mall_lat = []
mall_long = []

for i in range(0, len(list_of_shopping_mall)):
    query_address = list_of_shopping_mall[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)
    data_mall=json.loads(resp.content)
    
    if data_mall['found'] != 0:
        mall_name.append(query_address)
        mall_roadname.append(data_mall["results"][0]["ROAD_NAME"])
        mall_lat.append(data_mall["results"][0]["LATITUDE"])
        mall_long.append(data_mall["results"][0]["LONGITUDE"])

        print (str(query_address)+" ,Lat: "+data_mall['results'][0]['LATITUDE'] +" Long: "+data_mall['results'][0]['LONGITUDE'])

    else:
        print ("No Results")

# Store this information in a dataframe
mall_location = pd.DataFrame({
    'Mall': mall_name,
    'RoadName': mall_roadname,
    'Latitude': mall_lat,
    'Longitude': mall_long
})

100 AM ,Lat: 1.27458821795426 Long: 103.84347073661
313@Somerset ,Lat: 1.30138510214714 Long: 103.837684350436
Aperia ,Lat: 1.3104736675734 Long: 103.86431321816
Balestier Hill Shopping Centre ,Lat: 1.32616307866261 Long: 103.843741438467
Bugis Cube ,Lat: 1.2981408343975 Long: 103.855635339249
Bugis Junction ,Lat: 1.30011789343093 Long: 103.856191571652
Bugis+ ,Lat: 1.30095171530648 Long: 103.855172625542
Capitol Piazza ,Lat: 1.29307884763132 Long: 103.851261982149
Cathay Cineleisure Orchard ,Lat: 1.30152101873533 Long: 103.836429655016
City Gate ,Lat: 1.30231590504573 Long: 103.862331661034
City Square Mall ,Lat: 1.31142103107683 Long: 103.856624019991
CityLink Mall ,Lat: 1.2927777312893 Long: 103.854173501417
No Results
Duo ,Lat: 1.29953434891664 Long: 103.85840168774
Far East Plaza ,Lat: 1.30717698071189 Long: 103.833792932243
Funan ,Lat: 1.29134759697794 Long: 103.849989790085
Great World City ,Lat: 1.29342273662815 Long: 103.832022135776
HDB Hub ,Lat: 1.33205485190481 Long: 103.84

Alexandra Retail Centre ,Lat: 1.27414893629254 Long: 103.801399416665
HarbourFront Centre ,Lat: 1.26396991176004 Long: 103.820243325181
VivoCity ,Lat: 1.26429316783973 Long: 103.82230469365
321 Clementi ,Lat: 1.31200212030821 Long: 103.764986676365
Alexandra Central ,Lat: 1.28728320930592 Long: 103.805283367958
Anchorpoint ,Lat: 1.28861469030586 Long: 103.805009964592
Big Box ,Lat: 1.33191839981568 Long: 103.744920406659
Boon Lay Shopping Centre ,Lat: 1.34633552457315 Long: 103.712430373455
Fairprice Hub ,Lat: 1.32561767765205 Long: 103.678409741897
Gek Poh Shopping Centre ,Lat: 1.34874357136408 Long: 103.697732091001
Grantral Mall ,Lat: 1.31427043922294 Long: 103.765147487482
IMM ,Lat: 1.33489844010888 Long: 103.746734487309
JCube ,Lat: 1.33332312550425 Long: 103.740187071348
Jem ,Lat: 1.33305999269581 Long: 103.743503708169
Jurong Point ,Lat: 1.33945271661445 Long: 103.70668501289
No Results
Pioneer Mall ,Lat: 1.34169239214292 Long: 103.697174842118
Queensway Shopping Centre ,Lat: 1.

In [41]:
mrt_location.to_csv('./locations/mrt_location.csv',index=False)
mall_location.to_csv('./locations/mall_location.csv',index=False)

In [43]:
mrt_location

,MRT,Building,Latitude,Longitude
0,Jurong East MRT Station,JURONG EAST MRT STATION (EW24 / NS1),1.33315281585758,103.742286332403
1,Bukit Batok MRT Station,BUKIT BATOK MRT STATION (NS2),1.34903331201636,103.749566478309
2,Bukit Gombak MRT Station,BUKIT GOMBAK MRT STATION (NS3),1.35861159094192,103.751790910733
3,Choa Chu Kang MRT Station,CHOA CHU KANG MRT STATION (NS4),1.38536316540225,103.744370779756
4,Yew Tee MRT Station,YEW TEE MRT STATION (NS5),1.39753506936297,103.747405150236
...,...,...,...,...
115,Bedok North MRT Station,BEDOK NORTH MRT STATION (DT29),1.33474211664091,103.91797832995
116,Bedok Reservoir MRT Station,BEDOK RESERVOIR MRT STATION (DT30),1.33660782955099,103.932234623286
117,Tampines West MRT Station,TAMPINES WEST MRT STATION (DT31),1.34551530560119,103.938436971222
118,Tampines East MRT Station,TAMPINES EAST MRT STATION (DT33),1.35619148271544,103.9546344625


In [44]:
mall_location

,Mall,RoadName,Latitude,Longitude
0,100 AM,TRAS STREET,1.27458821795426,103.84347073661
1,313@Somerset,ORCHARD ROAD,1.30138510214714,103.837684350436
2,Aperia,KALLANG AVENUE,1.3104736675734,103.86431321816
3,Balestier Hill Shopping Centre,BALESTIER ROAD,1.32616307866261,103.843741438467
4,Bugis Cube,NORTH BRIDGE ROAD,1.2981408343975,103.855635339249
...,...,...,...,...
151,Taman Jurong Shopping Centre,YUNG SHENG ROAD,1.33484487416514,103.720462024278
152,The Clementi Mall,COMMONWEALTH AVENUE WEST,1.31489619639366,103.764423054189
153,The Star Vista,VISTA EXCHANGE GREEN,1.30697044038323,103.788420274115
154,Tiong Bahru Plaza,TIONG BAHRU ROAD,1.28645889472825,103.827015256194


In [50]:
print(f'{len(mrt_location)} mrt out of {len(list_of_mrt)} searched')
print(f'{len(mall_location)} malls out of {len(list_of_shopping_mall)} searched')

120 malls out of 120 searched
156 malls out of 171 searched


### getting data for all the house holds

In [77]:
from geopy.distance import geodesic, great_circle
from geopy.geocoders import GoogleV3
from geopy.geocoders import Nominatim

In [122]:
# Now - let's find grab the geolocation of each unit that was transacated using the same method. But hang on - that's a large dataset.. we can make it a bit more efficient. We know that there will be multiple units that will be transacted in the same HDB Apartment block. We can de-dup our dataframe and obtion the only unique addresses in our dataframe. 
# Let's combine the block and street name to form the address of our transacted unit.
df['address'] = df['block'] + ' ' + df['street_name'] + ' '+ 'SINGAPORE'



# Dedup Address List
df_dedup = df.drop_duplicates(subset='address', keep='first')
len(df_dedup)

# Next let's grab the unique addresses and create a list 
address_list = df_dedup['address'].tolist()
len(address_list)

9461

In [78]:
#!pip install geopy

There were 3 attempts to obtain geolocation data:
1. OneMap: After running through 105 street addresses program error code. saved to `hdb_location.csv`
2. OneMap: new method is used with only using TOA PAYOH town for comparisons. saved to `hdb_location2.csv`
3. MapQuest: New API used to get remainder of data. however abbreviated words like 'LOR' , 'CTRL' does not provide a result. saved to `hdb_location3.csv`

In [82]:
# This may take a while...

geolocator = Nominatim(user_agent="GoogleV3")

address = []
full_address = []
latitude = []
longitude = []
no_result = []

for row in range(len(address_list)):
    #formulate query string 
    try:
        add, (lat, lon) = geolocator.geocode(address_list[row])
        address.append(address_list[row])
        full_address.append(add)
        latitude.append(lat)
        longitude.append(lon)
    except:
        no_result.append(address_list[row])

df_coordinates = mall_location = pd.DataFrame({
    'address': address,
    'full_address': full_address,
    'Latitude': latitude,
    'Longitude': longitude
})

In [124]:
# save this third set of data 
df_coordinates.to_csv('./locations/hdb_location3.csv',index=False)

## Computing the nearest mrt, mall and names

In [157]:
# read all the data needed
mrt_location = pd.read_csv('./locations/mrt_location.csv')
mall_location = pd.read_csv('./locations/mall_location.csv')

# compile multiple location data to one dataframe
df_coordinates1 = pd.read_csv('./locations/hdb_location.csv')
df_coordinates1['address'] = df_coordinates1['blk_no'] + ' ' + df_coordinates1['road_name']
df_coordinates2 = pd.read_csv('./locations/hdb_location2.csv')
df_coordinates3 = pd.read_csv('./locations/hdb_location3.csv')
df_coordinates = pd.concat([df_coordinates1,df_coordinates2,df_coordinates3], ignore_index=True)

In [158]:
address =[]
d_mrt = []
n_mrt = []
d_mall = []
n_mall = []

for add,lat,lon in zip(df_coordinates.address,df_coordinates.latitude,df_coordinates.longitude):
    d_to_mrt = 99
    d_to_mall = 99
    temp = 0
    
    #loop for MRT and Mall to find the shortestdistance to address
    for mrt,mrt_lat,mrt_lon in zip(mrt_location.Building, mrt_location.Latitude, mrt_location.Longitude):
        temp = great_circle((lat, lon), (mrt_lat, mrt_lon)).km
        if d_to_mrt > temp:
            d_to_mrt = temp
            n_to_mrt = mrt
    for mall,mall_lat,mall_lon in zip(mall_location.Mall, mall_location.Latitude, mall_location.Longitude):
        temp = great_circle((lat, lon), (mall_lat, mall_lon)).km
        if d_to_mall > temp:
            d_to_mall = temp  
            n_to_mall = mall
    
    address.append(add)
    d_mrt.append(d_to_mrt)
    n_mrt.append(n_to_mrt)
    d_mall.append(d_to_mall)
    n_mall.append(n_to_mall)
    
df_coordinates['dist_mrt'] = d_mrt
df_coordinates['near_mrt'] = n_mrt
df_coordinates['dist_mall'] = d_mall
df_coordinates['near_mall'] = n_mall

In [159]:
# remove duplicates after compiling overlapping addresses
df_coordinates.drop_duplicates(subset=['address'],keep='last',inplace=True)

In [160]:
print(f'{len(df_coordinates)/len(address_list)*100:.0f}% of addresses were collected')

86% of addresses were collected


In [161]:
no_result = [x for x in df.address.unique() if x not in df_coordinates.address.unique()]

In [202]:
df_coordinates[df_coordinates.dist_mall == 99]

,latitude,longitude,blk_no,road_name,postal_code,address,full_address,block,street_name,dist_mrt,near_mrt,dist_mall,near_mall
3706,20.849793,106.67367,NaN,NaN,NaN,120 KIM TIAN PL,"SINGAPORE RESTAURANT, Tô Hiệu, Phần Lan, Quận ...",120,KIM TIAN PL,99.0,OUTRAM PARK MRT STATION,99.0,100 AM
3707,20.849793,106.67367,NaN,NaN,NaN,121 KIM TIAN PL,"SINGAPORE RESTAURANT, Tô Hiệu, Phần Lan, Quận ...",121,KIM TIAN PL,99.0,OUTRAM PARK MRT STATION,99.0,100 AM


There is a search that resulted in an address in Vietnam, this will be removed

In [204]:
df_coordinates =df_coordinates[df_coordinates.dist_mall != 99]

In [205]:
# save the location 
df_coordinates.to_csv('./locations/hdb_location_dist.csv',index=False)

### Compile HDB resale prices and distance data for visualisation

In [206]:
df.address = df.address.str.replace(' SINGAPORE', '')

In [217]:
df_final = pd.merge(df,df_coordinates,on=['block','street_name','address'],how='left')

In [218]:
df_final.shape

(849758, 26)

In [219]:
df_final.columns.to_list()

['town',
 'flat_type',
 'flat_model',
 'floor_area_sqm',
 'street_name',
 'resale_price',
 'month',
 'remaining_lease',
 'lease_commence_date',
 'storey_range',
 'block',
 'storey_lower',
 'storey_upper',
 'block_num',
 'storey_ave',
 'address',
 'latitude',
 'longitude',
 'blk_no',
 'road_name',
 'postal_code',
 'full_address',
 'dist_mrt',
 'near_mrt',
 'dist_mall',
 'near_mall']

In [220]:
df_final = df_final[['town',
                     'flat_type',
                     'flat_model',
                     'floor_area_sqm',
                     'resale_price',
                     'month',
                     'remaining_lease',
                     'storey_ave',
                     'address',
                     'latitude',
                     'longitude',
                     'full_address',
                     'dist_mrt',
                     'near_mrt',
                     'dist_mall',
                     'near_mall']]

In [221]:
df_final.to_csv('hdb_resale_dist.csv',index=False)